In [1]:
import importlib
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from datetime import datetime
from datetime import date
from datetime import timedelta

In [2]:
import PrjtCF_module as cf

# Outline
* 주거시설 등 개발과 동시에 분양을 하는 사업의 재무모델
* PF대출을 통해 토지비 및 공사비 등 개발사업 자금을 조달
* 금융비용은 PF대출금으로 충당
* 분양대금의 일부는 사업비로, 일부는 대출금 상환재원으로 사용
* PF대출금은 필요에 따라 한도대로 인출

# Input Index Data

In [3]:
# 기간 Index 설정
# prjt index : 사업기간 전 기간에 대한 index로 모델의 base가 되는 index
# cstrn index : 공사기간에 대한 index로 착공일로부터 준공일까지의 기간에 대한 index
# loan index : 대출기간에 대한 index로 대출 실행일로부터 대출 상환일 까지의 기간에 대한 index
# sales index : 분양기간에 대한 index로 분양개시일로부터 분양 종료일 까지의 기간 index
idx = cf.PrjtIndex(idxname=['prjt', 'cstrn', 'loan', 'sales'],
                   start=['2021-08', '2021-10', '2021-10', '2021-12'],
                   periods=[24+1, 18+1, 20+1, 16+1],
                   freq='M')

# 공정률 Series 설정 : cstrn index 기간 중 적용되는 기간별 공정률 설정
idx.prcs = Series(np.ones(len(idx.cstrn)) / len(idx.cstrn),
                  index=idx.cstrn.index)

# Input Financing Condition Data

In [4]:
equity = cf.Loan(idx, idx, amt_ntnl=10_000)

dct_loan = {}
tra = cf.Loan(idx, idx.loan, amt_ntnl=100_000,
              rate_fee = 0.01,
              rate_IR = 0.10)
dct_loan['tra'] = tra

loan = cf.Merge(dct_loan)
loan.tra = loan.dct['tra']

# Input Sales Data

In [5]:
dct_sales = {}

#### 분양상품A ####
# 분양매출 및 납입 일정
slsA = cf.Account(idx)
slsA.sls_amt = 150_000
slsA.csh_idx = idx.sales[[0, 3, 6, 9, 12, 16]]
slsA.csh_rate = Series([0.1, 0.1, 0.1, 0.1, 0.1, 0.5], 
                       index=slsA.csh_idx)
# 분양매출 가정
slsA.sls_rate = Series(np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]),
                       index=idx.sales[[0, 2, 4, 6, 7, 8, 9, 12, 14, 16]])
slsA.sls_plan = slsA.sls_amt * slsA.sls_rate

# Input Cost Data

In [6]:
dct_cost = {}

# 토지비: 최초 1회 지급
lnd = cf.Account(idx)
lnd.addscdd(idx.cstrn[0], 30_000)
dct_cost['lnd'] = lnd

# 공사비: 공정률에 따라 지급
cstrn = cf.Account(idx)
cstrn.addscdd(idx.cstrn.index, 50_000 * idx.prcs)
dct_cost['cstrn'] = cstrn

cost = cf.Merge(dct_cost)
cost.lnd = cost.dct['lnd']
cost.cstrn = cost.dct['cstrn']

# Execution Cash Flow

### 1) 사전 설정

In [7]:
# Make accounts
oprtg = cf.Account(idx) # 운영현금흐름의 입출을 위한 운영계좌
sales = cf.Account(idx) # 분양수입금의 입출을 위한 분양수입금 계좌
rpyacc = cf.Account(idx) # 상환자금 관리를 위한 대출금 상환 계좌

### 2) Cash Flow 실행

In [9]:
# Excute cash flow
for idxno in idx.index:
    # If it's initial date then set loan withdrawble.
    equity.set_wtdrbl_intldate(idxno)
    tra.set_wtdrbl_intldate(idxno)
        
    #### Sales : ####
    # 분양수입대금 sales schedule에 따라 운영계좌로 이체
    sls = cf.sls_mngmnt(idxno, slsA)
    sls.make_sls_plan()
    sls.rcv_slsamt(sales) # Receive sales amount on sales account
    # 분양수입금 분배
    intl_bal_end = sales.bal_end[idxno]
    amt_to_oprtg = intl_bal_end * 0.2
    amt_to_rpy = intl_bal_end * 0.8
    sales.send(idxno, amt_to_oprtg, oprtg)
    sales.send(idxno, amt_to_rpy, rpyacc)
    
    # cost instance 생성, 필요지출금액을 인자로 받아, 조달금액 계산 후 운영계좌 입금
    cst = cf.cst_mngmnt(idxno, cost, loan)
    wtdrw = cf.wtdrw_mngmnt(idxno, cst, oprtg)
    wtdrw.wtdrw_equity(equity)
    wtdrw.wtdrw_loan(tra)
    
    # 운영계좌에서 토지비, 공사비 등 각종 비용 지출
    cst.pay_oprtcst(oprtg)
    
    # 운영계좌에서 각종 금융비용 지출
    cst.pay_fnclcst(oprtg)
    
    # 대출금 상환 프로세스 진행
    repay = cf.repay_mngmnt(idxno, tra) # Loan instance class 생성
    repay.trsf_rpy(oprtg, rpyacc) # 상환 예정 금액을 상환계좌로 이체
    repay.rpy_ntnl(rpyacc) # 상환계좌에서 순서에 따라 상환금 인출
    
    # If it was maturity date then set back loan unwithdrawble.
    equity.setback_wtdrbl_mtrt(idxno)
    tra.setback_wtdrbl_mtrt(idxno)

In [10]:
# Merge Accounts
dct_acc = {'equity':equity,
           'tra_ntnl':tra.ntnl,
           'tra_fee':tra.fee,
           'tra_IR':tra.IR,
           'slsA':slsA,
           'cost_lnd':cost['lnd'],
           'cost_cstrn':cost['cstrn'],
           'oprtg':oprtg,
           'rpyacc':rpyacc}

# Print Result

In [11]:
# 현금흐름 종합
rslt_df = DataFrame({('equity.ntnl', 'amt_sub'):equity.ntnl.amt_sub[:],
                     ('slsA', 'amt_add'):slsA.amt_add[:],
                     ('slsA', 'amt_sub'):slsA.amt_sub[:],
                     ('tra.ntnl', 'amt_sub'):tra.ntnl.amt_sub[:],
                     ('tra.ntnl', 'amt_add'):tra.ntnl.amt_add[:],
                     ('tra.ntnl', 'bal_end'):tra.ntnl.bal_end[:],
                     ('tra.fee', 'amt_add'):tra.fee.amt_add[:],
                     ('tra.IR', 'amt_add'):tra.IR.amt_add[:],
                     ('cost_lnd', 'amt_add'):cost['lnd'].amt_add[:],
                     ('cost_cstrn', 'amt_add'):cost['cstrn'].amt_add[:],
                     ('oprtg', 'amt_add'):oprtg.amt_add[:],
                     ('oprtg', 'amt_sub'):oprtg.amt_sub[:],
                     ('oprtg', 'bal_end'):oprtg.bal_end[:],
                     ('rpyacc', 'amt_add'):rpyacc.amt_add[:],
                     ('rpyacc', 'amt_sub'):rpyacc.amt_sub[:],
                     ('rpyacc', 'bal_end'):rpyacc.bal_end[:]})
rslt_df.fillna(0).applymap(lambda x: f"{x:,.0f}")

equity.ntnl    slsA         tra.ntnl                  tra.fee  \
               amt_sub amt_add amt_sub  amt_sub amt_add  bal_end amt_add   
2021-08-31      10,000       0       0        0       0        0       0   
2021-09-30           0       0       0        0       0        0       0   
2021-10-31           0       0       0   23,632       0  -23,632   1,000   
2021-11-30           0       0       0    2,829       0  -26,460       0   
2021-12-31           0  15,000   1,500    2,552       0  -29,012       0   
2022-01-31           0       0       0    2,873       0  -31,886       0   
2022-02-28           0  15,000   1,500    2,597       0  -34,483       0   
2022-03-31           0       0   3,000    2,319       0  -36,802       0   
2022-04-30           0  15,000   3,000    2,338       0  -39,140       0   
2022-05-31           0       0       0    2,958       0  -42,098       0   
2022-06-30           0  15,000   9,000    1,182       0  -43,280       0   
2022-07-31           0  15,000   4,500    2,092       0  -45,372       0   
2022-08-31           0  15,000   4,500    2,110       0  -47,482       0   
2022-09-30           0  15,000  15,000       27       0  -47,509       0   
2022-10-31           0       0       0    3,027       0  -50,537       0   
2022-11-30           0       0       0    3,053       0  -53,590       0   
2022-12-31           0  15,000  18,000        0       0  -53,590       0   
2023-01-31           0       0       0    2,556       0  -56,146       0   
2023-02-28           0  15,000   7,500    1,599       0  -57,745       0   
2023-03-31           0       0       0    3,113       0  -60,858       0   
2023-04-30           0  15,000  82,500        0       0  -60,858       0   
2023-05-31           0       0       0        0       0  -60,858       0   
2023-06-30           0       0       0        0  60,858        0       0   
2023-07-31           0       0       0        0       0        0       0   
2023-08-31           0       0       0        0       0        0       0   

            tra.IR cost_lnd cost_cstrn   oprtg                   rpyacc  \
           amt_add  amt_add    amt_add amt_add amt_sub  bal_end amt_add   
2021-08-31       0        0          0  10,000       0   10,000       0   
2021-09-30       0        0          0       0       0   10,000       0   
2021-10-31       0   30,000      2,632  23,632  33,632        0       0   
2021-11-30     197        0      2,632   2,829   2,829        0       0   
2021-12-31     221        0      2,632   2,852   2,852        0   1,200   
2022-01-31     242        0      2,632   2,873   2,873        0       0   
2022-02-28     266        0      2,632   2,897   2,897        0   1,200   
2022-03-31     287        0      2,632   2,919   2,919        0   2,400   
2022-04-30     307        0      2,632   2,938   2,938        0   2,400   
2022-05-31     326        0      2,632   2,958   2,958        0       0   
2022-06-30     351        0      2,632   2,982   2,982        0   7,200   
2022-07-31     361        0      2,632   2,992   2,992        0   3,600   
2022-08-31     378        0      2,632   3,010   3,010        0   3,600   
2022-09-30     396        0      2,632   3,027   3,027        0  12,000   
2022-10-31     396        0      2,632   3,027   3,027        0       0   
2022-11-30     421        0      2,632   3,053   3,053        0       0   
2022-12-31     447        0      2,632   3,600   3,078      522  14,400   
2023-01-31     447        0      2,632   2,556   3,078        0       0   
2023-02-28     468        0      2,632   3,099   3,099        0   6,000   
2023-03-31     481        0      2,632   3,113   3,113        0       0   
2023-04-30     507        0      2,632  16,500   3,139   13,361  66,000   
2023-05-31     507        0          0       0     507   12,854       0   
2023-06-30     507        0          0       0  61,365  -48,511  60,858   
2023-07-31       0        0          0       0       0  -48,511       0   
2023-08-

In [12]:
# 현금흐름 요약
rslt_df = DataFrame({('equity.ntnl', 'amt_sub'):equity.ntnl.amt_sub[:],
                     ('slsA', 'amt_add'):slsA.amt_add[:],
                     ('slsA', 'amt_sub'):slsA.amt_sub[:],
                     ('tra.ntnl', 'amt_sub'):tra.ntnl.amt_sub[:],
                     ('tra.ntnl', 'amt_add'):tra.ntnl.amt_add[:],
                     ('oprtg', 'amt_add'):oprtg.amt_add[:],
                     ('tra.fee', 'amt_add'):tra.fee.amt_add[:],
                     ('tra.IR', 'amt_add'):tra.IR.amt_add[:],
                     ('cost_lnd', 'amt_add'):cost['lnd'].amt_add[:],
                     ('cost_cstrn', 'amt_add'):cost['cstrn'].amt_add[:],
                     ('oprtg', 'amt_sub'):oprtg.amt_sub[:],
                     ('oprtg', 'bal_end'):oprtg.bal_end[:]})
rslt_df.fillna(0).applymap(lambda x: f"{x:,.0f}")

equity.ntnl    slsA         tra.ntnl           oprtg tra.fee  \
               amt_sub amt_add amt_sub  amt_sub amt_add amt_add amt_add   
2021-08-31      10,000       0       0        0       0  10,000       0   
2021-09-30           0       0       0        0       0       0       0   
2021-10-31           0       0       0   23,632       0  23,632   1,000   
2021-11-30           0       0       0    2,829       0   2,829       0   
2021-12-31           0  15,000   1,500    2,552       0   2,852       0   
2022-01-31           0       0       0    2,873       0   2,873       0   
2022-02-28           0  15,000   1,500    2,597       0   2,897       0   
2022-03-31           0       0   3,000    2,319       0   2,919       0   
2022-04-30           0  15,000   3,000    2,338       0   2,938       0   
2022-05-31           0       0       0    2,958       0   2,958       0   
2022-06-30           0  15,000   9,000    1,182       0   2,982       0   
2022-07-31           0  15,000   4,500    2,092       0   2,992       0   
2022-08-31           0  15,000   4,500    2,110       0   3,010       0   
2022-09-30           0  15,000  15,000       27       0   3,027       0   
2022-10-31           0       0       0    3,027       0   3,027       0   
2022-11-30           0       0       0    3,053       0   3,053       0   
2022-12-31           0  15,000  18,000        0       0   3,600       0   
2023-01-31           0       0       0    2,556       0   2,556       0   
2023-02-28           0  15,000   7,500    1,599       0   3,099       0   
2023-03-31           0       0       0    3,113       0   3,113       0   
2023-04-30           0  15,000  82,500        0       0  16,500       0   
2023-05-31           0       0       0        0       0       0       0   
2023-06-30           0       0       0        0  60,858       0       0   
2023-07-31           0       0       0        0       0       0       0   
2023-08-31           0       0       0        0       0       0       0   

            tra.IR cost_lnd cost_cstrn   oprtg           
           amt_add  amt_add    amt_add amt_sub  bal_end  
2021-08-31       0        0          0       0   10,000  
2021-09-30       0        0          0       0   10,000  
2021-10-31       0   30,000      2,632  33,632        0  
2021-11-30     197        0      2,632   2,829        0  
2021-12-31     221        0      2,632   2,852        0  
2022-01-31     242        0      2,632   2,873        0  
2022-02-28     266        0      2,632   2,897        0  
2022-03-31     287        0      2,632   2,919        0  
2022-04-30     307        0      2,632   2,938        0  
2022-05-31     326        0      2,632   2,958        0  
2022-06-30     351        0      2,632   2,982        0  
2022-07-31     361        0      2,632   2,992        0  
2022-08-31     378        0      2,632   3,010        0  
2022-09-30     396        0      2,632   3,027        0  
2022-10-31     396        0      2,632   3,027        0  
2022-11-30     421        0      2,632   3,053        0  
2022-12-31     447        0      2,632   3,078      522  
2023-01-31     447        0      2,632   3,078        0  
2023-02-28     468        0      2,632   3,099        0  
2023-03-31     481        0      2,632   3,113        0  
2023-04-30     507        0      2,632   3,139   13,361  
2023-05-31     507        0          0     507   12,854  
2023-06-30     507        0          0  61,365  -48,511  
2023-07-31       0        0          0       0  -48,511  
2023-08-31       0        0          0       0  -48,511

In [14]:
slsA.df

,add_scdd,sub_scdd,bal_strt,amt_add,amt_sub,bal_end
2021-08-31,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-30,0.0,0.0,0.0,0.0,0.0,0.0
2021-10-31,0.0,0.0,0.0,0.0,0.0,0.0
2021-11-30,0.0,0.0,0.0,0.0,0.0,0.0
2021-12-31,0.0,15000.0,0.0,15000.0,1500.0,13500.0
2022-01-31,0.0,0.0,13500.0,0.0,0.0,13500.0
2022-02-28,0.0,0.0,13500.0,15000.0,1500.0,27000.0
2022-03-31,0.0,15000.0,27000.0,0.0,3000.0,24000.0
2022-04-30,0.0,0.0,24000.0,15000.0,3000.0,36000.0
2022-05-31,0.0,0.0,36000.0,0.0,0.0,36000.0


In [16]:
slsA.df

,add_scdd,sub_scdd,bal_strt,amt_add,amt_sub,bal_end
2021-08-31,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-30,0.0,0.0,0.0,0.0,0.0,0.0
2021-10-31,0.0,0.0,0.0,0.0,0.0,0.0
2021-11-30,0.0,0.0,0.0,0.0,0.0,0.0
2021-12-31,0.0,15000.0,0.0,15000.0,1500.0,13500.0
2022-01-31,0.0,0.0,13500.0,0.0,0.0,13500.0
2022-02-28,0.0,0.0,13500.0,15000.0,1500.0,27000.0
2022-03-31,0.0,15000.0,27000.0,0.0,3000.0,24000.0
2022-04-30,0.0,0.0,24000.0,15000.0,3000.0,36000.0
2022-05-31,0.0,0.0,36000.0,0.0,0.0,36000.0
